<a href="https://colab.research.google.com/github/sileeee/python-data_science_study/blob/master/datascience_week4(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Ensemble
: Ensemble(앙상블)이란 여러 머신러닝 모델을 연결하여 더 강력한 모델을 만드는 기법
- 정확도가 상승, 처리속도 느려짐
- (일반적으로는 혼자 판단하기 보다 각기 전문 영역을 가진 여러명의 전문가들의 의견을 들어 종합하는게 더 합리적인 의사결정)

### Random Forest
: Random Forest가 여러개의 Decision Tree를 만들고 연결하여 결과를 취합한 후 평균내어 성능을 높인 모델이다. 즉 Ensemble의 원리를 이용한 방법.
- Decision Tree는 꽤 좋은 모델이라 여러 상황에서도 예측을 잘 하는 편이지만, 일부 데이터에 대해 overfitting하는 경향이 있다. 그러한 원리, 문제를 해결하는 방법.

### Overfitting
: Overfitting(과적합)이란 학습 데이터를 과하게 학습한 나머지 학습 상황에서는 오차가 줄지만 실제 데이터들에 대해 오차가 증가하는 현상, 너무 지나치게 훈련 데이터를 맹신해서 실제 예측이 자주 빗나가는 현상<br/>
(ex) 머신러닝에서) Train Accuracy가 100%가 나오는데 실제 테스트는 현저하게 떨어지는 50%가 나온다.
(ex) 일상에서) 볼링공에 머리를 한번 맞아 봤는데 전혀 아프지 않았다. 볼링공으로 사람 머리를 때리는건 하나도 위험하지 않구나.

### Decision Tree를 Ensemble 해보기

- 여러개의 Decision Tree 만들기

(지난주 코드에 이어서)



(1) 세개의 트리 구성 및 정확도 확인

In [ ]:
from sklearn.tree import DecisionTreeClassifier

tree1 = DecisionTreeClassifier()
tree1.fit(x_train, y_train)

print('training set accuracy:', tree1.score(x_train, y_train))

tree2 = DecisionTreeClassifier()
tree2.fit(x_train, y_train)

print('training set accuracy:', tree2.score(x_train, y_train))

tree3 = DecisionTreeClassifier()
tree3.fit(x_train, y_train)

print('training set accuracy:', tree3.score(x_train, y_train))

In [ ]:
# 앞 쪽 100개만 validation set으로 지정하기
x_valid = x_train[0:100]
y_valid = y_train[0:100]

# 앞 100개 데이터를 제외한 남은 부분을 다시 trainset으로 재정의
x_train = x_train[100:]
y_train = y_train[100:]

In [ ]:
# 100개의 데이터를 제외한 x_train, y_train를 다시 훈련시켜줍니다.
tree1.fit(x_train, y_train)

print('training set accuracy:', tree1.score(x_train, y_train))
print('validation set accuracy:', tree1.score(x_valid, y_valid))

tree2 = DecisionTreeClassifier()
tree2.fit(x_train, y_train)

print('training set accuracy:', tree2.score(x_train, y_train))
print('validation set accuracy:', tree2.score(x_valid, y_valid))

tree3 = DecisionTreeClassifier()
tree3.fit(x_train, y_train)

print('training set accuracy:', tree3.score(x_train, y_train))
print('validation set accuracy:', tree3.score(x_valid, y_valid))

(2) 여러개의 Decision Tree Ensemble 하기

In [ ]:
prediction1 = tree1.predict(test)
prediction2 = tree2.predict(test)
prediction3 = tree3.predict(test)

ensemble = (prediction1 + prediction2 + prediction3) / 3
ensemble[ensemble > 0.5] = 1
ensemble[ensemble <= 0.5] = 0

# 평균을 내는 작업으로 인해 survived 데이터가 1,0에서 소수형태로 바뀜
# 생존여부는 1,0으로 표시해야 하므로 다시 정수 형태로 변환
ensemble = ensemble.astype(int)

ensemble

- tree1의 submit파일 만들기

In [ ]:
submit = pd.DataFrame({
    'PassengerId': df_test['PassengerId'],
    'Survived': ensemble
})

submit.to_csv('submit_ensemble.csv', index=False)

### Iris문제 Feature Engineering
- Iris 데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('data/iris.csv')
df.head(10)

In [ ]:
df.info()

- Feature Engineering and Visualization

1)꽃받침 길이, 너비에 따른 종 분포도 보기

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set()

In [ ]:
sns.catplot(data=df, x='SepalLengthCm', y='SepalWidthCm', hue='Species', aspect=2)

2) 꽃잎의 길이, 너비에 따른 종 분포도 보기

In [ ]:
sns.catplot(data=df, x='PetalLengthCm', y='PetalWidthCm', hue='Species', aspect=2)

- 특징 생성 - Feature Engineering with Pandas

1) kdeplot으로 꽃받침 길이별 밀집 수준 파악

In [ ]:
facet = sns.FacetGrid(df, hue="Species", aspect=2)
facet.map(sns.kdeplot, 'SepalLengthCm')
facet.add_legend()

plt.show()

2) box plot으로 종별 꽃받침 길이 범위 파악

In [ ]:
sns.catplot(kind='box', data=df, x='Species', y='SepalLengthCm', aspect=2)

3) violin plot - x: 종, y: 꽃받침 길이

In [ ]:
sns.violinplot(data=df, x='Species', y='SepalLengthCm')

4) violin plot - x: 종, y: 꽃받침 너비

In [ ]:
sns.violinplot(data=df, x='Species', y='SepalWidthCm')

5) violin plot - x: 종, y: 꽃잎 길이

In [ ]:
sns.violinplot(data=df, x='Species', y='PetalLengthCm')

6) violin plot - x: 종, y: 꽃잎 너비

### 바이올린 플롯
Violin Plot은 값 범위와 밀집정도를 시각적으로 동시에 확인할 수 있는 플롯

In [ ]:
sns.violinplot(data=df, x='Species', y='PetalWidthCm')

- 카테고리 플랏은 값의 범위랑 그 부분에 데이터가 몇개나 있는지 알기 어렵다.
그래서, 박스플롯과 kdeplot을 사용한다. 밀집도와 값의 범위를 쉽게 확인할 수 있다.
이 두가지 방식을 혼합할 수 있는 플롯은 값의 범위(최소, 최대, 이상값)와 밀집 정도까지 시각적으로 한번에 알 수 있다. => 바이올린 플롯